<a href="https://colab.research.google.com/github/fmathews11/Football_Field_Goals/blob/main/Predicting_Field_Goals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, re
from pandas.api.types import is_string_dtype
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
pd.set_option('display.max_columns', None)
from google.colab import output
output.clear()
df = pd.read_csv('https://raw.githubusercontent.com/fmathews11/Football_Field_Goals/main/historical_field_goal_data.csv')
#print(*((idx,i) for idx,i in enumerate(df.columns)),sep = '\n')
df = df.drop([df.columns[i] for i in [0,11,12,31]],axis =1)
df.isna().sum()

week                      0
away                      0
home                      0
stadium                   0
temperature            2252
humidity               2499
wind_speed             2387
wind_direction         2405
condition                36
surface                   0
away_score                0
home_score                0
off                       0
def                       0
quarter                   0
minutes                   0
seconds                   0
offense_score             0
defense_score             0
timeouts_offense          0
timeouts_defense          0
down                      0
yards_to_go               0
yards_from_own_goal       0
points                    0
penalty                   0
fumble                    0
block                     0
kicker_id                 0
kicker                    0
distance                  0
made                      0
dtype: int64

In [21]:
pd.Series({col:df[col].nunique() for col in df if is_string_dtype(df[col])}).index.tolist()

['away',
 'home',
 'stadium',
 'wind_direction',
 'condition',
 'surface',
 'off',
 'def',
 'kicker']

In [22]:
#Filtering by stadiums with a minimum of 50 kicks recorded
df = df[df.stadium.isin(df.groupby('stadium').size().sort_values()[lambda x: x > 50].index.tolist())]

#Imputing the wind direction with the most common value by stadium
df['wind_direction_mode'] = df.groupby('stadium').wind_direction.transform(lambda x: x.value_counts().index[0])

IndexError: ignored

In [23]:
columns_to_encode = [i for i in pd.Series({col:df[col].nunique() for col in df if is_string_dtype(df[col])}).index]
le_dict = {key:LabelEncoder() for key in columns_to_encode}
for col in le_dict.keys():
  print(col)
  df[col] = le_dict[col].fit_transform(df[col])
df

away
home
stadium
wind_direction


TypeError: ignored

In [18]:
#Filtering by stadiums with a minimum of 50 kicks recorded
df = df[df.stadium.isin(df.groupby('stadium').size().sort_values()[lambda x: x > 50].index.tolist())]

df.wind_direction = df.wind_direction.fillna(df.wind_direction_mode)
df.drop('wind_direction_mode',axis = 1,inplace= True)
df

AttributeError: ignored

In [125]:


def assign_month(x):
  if x.week in [1,2,3]: return 9
  elif x.week in [4,5,6,7,8]: return 10
  elif x.week in [9,10,11,12]: return 11
  elif x.week in [13,14,15,16]: return 12
  elif x.week in [17,18,19,20]:return 1
  else: return 2
df['month'] = df.apply(lambda x: assign_month(x),axis = 1)
df.temperature = df.temperature.fillna(df.groupby(['stadium','month']).temperature.transform("mean"))

In [118]:
def assign_month(x):
  if x.week in [1,2,3]: return 9
  elif x.week in [4,5,6,7,8]: return 10
  elif x.week in [9,10,11,12]: return 11
  elif x.week in [13,14,15,16]: return 12
  elif x.week in [17,18,19,20]:return 1
  else: return 2
df['month'] = df.apply(lambda x: assign_month(x),axis = 1)
df

,week,away,home,stadium,temperature,humidity,wind_speed,wind_direction,condition,surface,away_score,home_score,off,def,quarter,minutes,seconds,offense_score,defense_score,timeouts_offense,timeouts_defense,down,yards_to_go,yards_from_own_goal,points,penalty,fumble,block,kicker_id,kicker,distance,made
0,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,23,8,2,9,14,0,0,3,3,4,4,96,3,0,0,0,2886,48,22,1
1,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,8,23,3,0,45,14,10,2,3,4,15,85,3,0,0,0,4452,16,33,1
2,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,2,5,36,14,7,1,3,4,4,83,3,0,0,0,2820,68,35,1
3,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,13,5,2,0,6,14,24,1,1,2,10,81,0,0,0,0,2888,0,37,0
4,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,3,10,7,31,14,3,3,4,6,92,3,0,0,0,2820,68,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,20,3,15,2,40.0,70.0,7.0,6,4,5,24,38,3,15,4,3,19,21,38,3,3,4,28,67,3,0,0,0,54578,84,51,1
9282,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,1,5,14,0,0,3,3,4,11,69,3,0,0,0,53628,28,49,1
9283,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,2,1,4,3,14,3,2,4,6,86,3,0,0,0,53628,28,34,1
9284,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,3,11,31,6,21,3,3,4,7,66,3,0,0,0,53628,28,52,1
